In [2]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
from notebookutils import mssparkutils as mu
import pandas as pd

StatementMeta(, e68d3f57-9671-47c1-9191-0b29732d9133, 4, Finished, Available, Finished)

In [3]:
# Crear estructura de esquemas en el lakehouse para almacenar nuestras tablas procesadas
spark = SparkSession.builder.getOrCreate()
try:
    #creando carpetas para datos procesados, esquema medallion. Premisa: Capa Bronze existente, Pipeline Copy Data
    spark.sql("CREATE SCHEMA IF NOT EXISTS bronze")
    spark.sql("CREATE SCHEMA IF NOT EXISTS silver")
    spark.sql("CREATE SCHEMA IF NOT EXISTS gold")
    print("Esquemas creados")
except Exception as e:
    print(f"Error al crear esquemas: {str(e)}")

StatementMeta(, e68d3f57-9671-47c1-9191-0b29732d9133, 5, Finished, Available, Finished)

Carpetas creadas


In [5]:
# Leer los archivos CSV con los esquemas definidos
# Este comando lee los archivos CSV utilizando los esquemas definidos anteriormente

try:
    # Leer archivo de brokers
    df_brokers = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load("Files/raw/brokers.csv")
    
    # Leer archivo de campaigns
    df_campaigns = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load("Files/raw/campaigns.csv")
    
    # Leer archivo de clients
    df_clients = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load("Files/raw/clients.csv")
    
    # Leer archivo de leads
    df_leads = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load("Files/raw/leads.csv")
    
    # Leer archivo de projects
    df_projects = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load("Files/raw/projects.csv")
    
    # Leer archivo de properties
    df_properties = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load("Files/raw/properties.csv")
    
    # Leer archivo de sales
    df_sales = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load("Files/raw/sales.csv")
    
    print("Archivos CSV leídos correctamente de la capa raw")
except Exception as e:
    print(f"Error al leer archivos: {str(e)}")

StatementMeta(, e68d3f57-9671-47c1-9191-0b29732d9133, 7, Finished, Available, Finished)

Archivos CSV leídos correctamente de la capa raw


In [8]:
# Realizar transformaciones en los dataframes
# Aplicamos transformaciones básicas añadir metadata a los df

# Transformación de brokers: 
df_brokers_raw = df_brokers.withColumn("_fecha_carga", F.current_timestamp()) \
                 .withColumn("_fuente", F.lit("ADSL")) \
                 .withColumn("_usuario", F.lit("etl_user"))

# Transformación de campaigns: 
df_campaigns_raw = df_campaigns.withColumn("_fecha_carga", F.current_timestamp()) \
                 .withColumn("_fuente", F.lit("ADSL")) \
                 .withColumn("_usuario", F.lit("etl_user"))

# Transformación de clients: 
df_clients_raw = df_clients.withColumn("_fecha_carga", F.current_timestamp()) \
                 .withColumn("_fuente", F.lit("ADSL")) \
                 .withColumn("_usuario", F.lit("etl_user"))

# Transformación de leads: 
df_leads_raw = df_leads.withColumn("_fecha_carga", F.current_timestamp()) \
                 .withColumn("_fuente", F.lit("ADSL")) \
                 .withColumn("_usuario", F.lit("etl_user"))

# Transformación de leads: 
df_projects_raw = df_projects.withColumn("_fecha_carga", F.current_timestamp()) \
                 .withColumn("_fuente", F.lit("ADSL")) \
                 .withColumn("_usuario", F.lit("etl_user"))

# Transformación de properties: Añadir columna precio del m2
df_properties_raw = df_properties.withColumn("_fecha_carga", F.current_timestamp()) \
                 .withColumn("_fuente", F.lit("ADSL")) \
                 .withColumn("_usuario", F.lit("etl_user"))

# Transformación de sales: Extraer el año, trimestre, mes, día y periodo de la venta
df_sales_raw = df_sales.withColumn("_fecha_carga", F.current_timestamp()) \
                 .withColumn("_fuente", F.lit("ADSL")) \
                 .withColumn("_usuario", F.lit("etl_user"))

print("Transformaciones aplicadas correctamente")

StatementMeta(, e68d3f57-9671-47c1-9191-0b29732d9133, 10, Finished, Available, Finished)

Transformaciones aplicadas correctamente


In [12]:
# Guardamos los dataframes como tablas en el Lakehouse actual en el esquema bronze
df_brokers_raw.write.format("delta").mode("overwrite").saveAsTable("bronze.brokers")
df_campaigns_raw.write.format("delta").mode("overwrite").saveAsTable("bronze.campaigns")
df_clients_raw.write.format("delta").mode("overwrite").saveAsTable("bronze.clients")
df_leads_raw.write.format("delta").mode("overwrite").saveAsTable("bronze.leads")
df_projects_raw.write.format("delta").mode("overwrite").saveAsTable("bronze.projects")
df_properties_raw.write.format("delta").mode("overwrite").saveAsTable("bronze.properties")
df_sales_raw.write.format("delta").mode("overwrite").saveAsTable("bronze.sales")

StatementMeta(, e68d3f57-9671-47c1-9191-0b29732d9133, 14, Finished, Available, Finished)